In [1]:
#this imports the necessary modules
from music21 import *
import pickle, time, os
from sklearn.externals import joblib

In [ ]:
unparsed = '/home/daniel/Desktop/ElvisVictoria/'
all_parsed = []
for root, dirs, files in os.walk(unparsed):
    for file_name, x in zip(files, range(len(files))):
#                 print(file_name, '...', x + 1, 'of', len(files)+1, '...')
        if file_name.endswith('.krn') or file_name.endswith('.mid'):
            #                     print('it is a krn')
            path = os.path.join(root, file_name)
            st = time.time()
            print('parsing', file_name)
            all_parsed.append(converter.parse(path))
all_percents = []
all_keys = []
for song in all_parsed:
    aa = song.flat.getElementsByClass(note.Note)

    all_notes = []
    pitch_frequency_data = []
    for x in aa:
        all_notes.append(x.pitch.pitchClass)
    bass_note = all_notes[-1]
    all_notes = [(x-bass_note)%12 for x in all_notes]
    note_data = {}
    song_percents = []
    note_counter = 0
    for xx in range(12):
        note_data[xx] = 0
    for xx in all_notes:
        note_data[xx] += 1
    for xx in range(12):
        song_percents.append(note_data[xx] / len(all_notes) * 100)
    all_percents.append(song_percents)
    all_keys.append(bass_note)

In [ ]:
print(len(all_parsed))
joblib.dump((all_percents, all_keys), 'pickles/victoria_notes.pkl', compress=9)

In [2]:
#A is the path to the folder that holds all of the songs
# A = 'kern_files/wtc/'
A = corpus.getComposer('monteverdi') #ignore this for now


#these match the key or mode of the song with the appropriate label
tonal_major_dict = {(0, 0): 'CM', (1, 7): 'GM', (2, 2): 'DM', (3, 9): 'AM', (4, 4): 'EM', (5, 11): 'BM', (6, 6): 'F♯M',
               (-1, 5): 'FM', (-2, 10): 'B♭M', (-3, 3): 'E♭M', (-4, 8): 'A♭M', (-5, 1): 'D♭M', (-6, 6): 'G♭M'}
tonal_minor_dict = {(0, 9): 'am', (1, 4): 'em', (2, 11): 'bm', (3, 6): 'f♯m', (4, 1): 'c♯m', (5, 8): 'g♯m',
                    (6, 3): 'd♯m', (-1, 2): 'dm', (-2, 7): 'gm', (-3, 0): 'cm', (-4, 5): 'fm', (-5, 10): 'b♭m',
                    (-6, 3): 'e♭m'}
modal_major = [('f', 5), ('n', 0), ('f', 0), ('n', 7), ('f', 10), ('n', 5),
               (-1, 5), (0, 0), (-1, 0), (0, 7), (-1, 10), (0, 5)]
modal_minor = [('n', 2), ('n', 9), ('f', 2), ('f', 7), ('n', 4),
               (0, 2), (0, 9), (-1, 2), (-1, 7), (0, 4)]

In [3]:
# this function converts .krn or .xml files into the format we need
def data_converter(unparsed, modality, corpus_name):
    pitch_frequency_data = []
    all_parsed = []
    all_keys = []
    all_chords = []
    all_titles = []

    def parser(parsed_songs):
        for x in all_parsed:
            sst = time.time()
            if type(x) is stream.Score:
    #             print('we have a score!')
                song_chords = []
                crd = x.chordify().flat.getElementsByClass(chord.Chord)
                if len(x.flat.getElementsByClass(key.KeySignature)) > 0:
                    key_sig = x.flat.getElementsByClass(key.KeySignature)[0].sharps
                    bass_note = crd[-1].bass().pitchClass
                    modal_type = []
                    if modality == 'tonal':
                        for c in tonal_major_dict.keys():
                            modal_type.append(c)
                        for c in tonal_minor_dict.keys():
                            modal_type.append(c)
                            #                 modal_type.append(c for c in tonal_minor_dict.keys())
                    elif modality == 'modal':
                        for c in modal_major:
                            modal_type.append(c)
                        for c in modal_minor:
                            modal_type.append(c)
                            #             print(modal_type, (key_sig, bass_note))
                    if (key_sig, bass_note) in modal_type:
                        print((key_sig, bass_note), 'is in there')
                        for c in crd:
                            song_chord = []
                            note_bass = (c.bass().pitchClass - bass_note) % 12
                            song_chord.append(note_bass)
                            song_chord.append(sorted(set([(n.pitchClass - bass_note - note_bass) % 12 for n in c])))
                            song_chords.append(song_chord)
                        all_chords.append(song_chords)
                        all_keys.append((key_sig, bass_note))
                        try:
                            all_titles.append(x.metadata.title)
                        except AttributeError:
                            all_titles.append('notitle')
                        
                    # this is for pitch class frequency data
                        all_notes = []
                        note_data = {}
                        song_percents = []
                        note_counter = 0
                        for xx in crd:
                            for yy in xx:
                                all_notes.append((yy.pitchClass - bass_note) % 12)
                                note_counter += 1
                        for xx in range(12):
                            note_data[xx] = 0
                        for xx in all_notes:
                            note_data[xx] += 1
                        for xx in range(12):
                            song_percents.append(note_data[xx] / len(all_notes) * 100)
                        pitch_frequency_data.append(song_percents)
                        try:
                            print('completed', x.metadata.title, 'in ',time.time()-sst, ' seconds')
                        except AttributeError:
                            print('completed', 'notitle', 'in ',time.time()-sst, ' seconds')
                        
                    #                 all_corpus_labels.append((the_key[0].sharps, bass_note))

            elif type(x) is stream.Opus:
                print('we have an Opus!')
                for y in x:
                    song_chords = []
                    crd = y.chordify().flat.getElementsByClass(chord.Chord)
                    if len(x.flat.getElementsByClass(key.KeySignature)) > 0:
                        key_sig = y.flat.getElementsByClass(key.KeySignature)[0].sharps
                        bass_note = crd[-1].bass().pitchClass
                        modal_type = []
                        if modality == 'tonal':
                            for c in tonal_major_dict.keys():
                                modal_type.append(c)
                            for c in tonal_minor_dict.keys():
                                modal_type.append(c)
                                #                     modal_type.append(c for c in tonal_minor_dict.keys())
                        elif modality == 'modal':
                            for c in modal_major:
                                modal_type.append(c)
                            for c in modal_minor:
                                modal_type.append(c)
                        if (key_sig, bass_note) in modal_type:
                            print((key_sig, bass_note), 'is in there')
                            for c in crd:
                                song_chord = []
                                note_bass = (c.bass().pitchClass - bass_note) % 12
                                song_chord.append(note_bass)
                                song_chord.append(sorted(set([(n.pitchClass - bass_note - note_bass) % 12 for n in c])))
                                song_chords.append(song_chord)
                            all_chords.append(song_chords)
                            all_keys.append((key_sig, bass_note))
                            try:
                                all_titles.append(x.metadata.title)
                            except AttributeError:
                                all_titles.append('notitle')
                        # this is for pitch class frequency data
                            all_notes = []
                            note_data = {}
                            song_percents = []
                            note_counter = 0
                            for xx in crd:
                                for yy in xx:
                                    all_notes.append((yy.pitchClass - bass_note) % 12)
                                    note_counter += 1
                            for xx in range(12):
                                note_data[xx] = 0
                            for xx in all_notes:
                                note_data[xx] += 1
                            for xx in range(12):
                                song_percents.append(note_data[xx] / len(all_notes) * 100)
                            pitch_frequency_data.append(song_percents)
                            print(len(pitch_frequency_data), len(all_keys))

    start = time.time()
    if type(unparsed) is str:

        for root, dirs, files in os.walk(unparsed):
            for file_name, x in zip(files, range(len(files))):
#                 print(file_name, '...', x + 1, 'of', len(files)+1, '...')
                if file_name.endswith('.krn') or file_name.endswith('.mid'):
                    #                     print('it is a krn')
                    path = os.path.join(root, file_name)
                    st = time.time()
                    print('parsing', file_name)
                    all_parsed.append(converter.parse(path))
#                     parser(path)
                    fin = time.time()
                    print('parsed', len(all_parsed), 'of', len(unparsed), 'in', fin - st, 'seconds')
    else:
        for x in unparsed:
            st = time.time()
#             print('parsing', x)
            all_parsed.append(converter.parse(x))
#             parser(x)
            fin = time.time()
            print('parsed', len(all_parsed), 'of', len(unparsed), 'in', fin - st, 'seconds')
    print('entire process took:', time.time() - start, 'seconds')
    parser(all_parsed)
    joblib.dump((all_chords, all_keys), 'pickles/%s_continuo.pkl' % corpus_name, compress=9)
    joblib.dump((pitch_frequency_data, all_keys), 'pickles/%s_notes.pkl' % corpus_name, compress=9)

    print('ALL DONE!!! :) :) :)')
    
def chordify_maker(unparsed, modality, corpus_name):
    all_chords = []

    def parser(parsed_songs):
        for x in all_parsed:
            if type(x) is stream.Score:
                sst = time.time()
    #             print('we have a score!')
                song_chords = []
                crd = x.chordify().flat.getElementsByClass(chord.Chord)
                if len(x.flat.getElementsByClass(key.KeySignature)) > 0:
                    key_sig = x.flat.getElementsByClass(key.KeySignature)[0].sharps
                    bass_note = crd[-1].bass().pitchClass
                    modal_type = []
                    if modality == 'tonal':
                        for c in tonal_major_dict.keys():
                            modal_type.append(c)
                        for c in tonal_minor_dict.keys():
                            modal_type.append(c)
                            #                 modal_type.append(c for c in tonal_minor_dict.keys())
                    elif modality == 'modal':
                        for c in modal_major:
                            modal_type.append(c)
                        for c in modal_minor:
                            modal_type.append(c)
                            #             print(modal_type, (key_sig, bass_note))
                    if (key_sig, bass_note) in modal_type:
                        for c in crd:
                            song_chord = []
                            note_bass = (c.bass().pitchClass - bass_note) % 12
                            song_chord.append(note_bass)
                            song_chord.append(sorted(set([(n.pitchClass - bass_note - note_bass) % 12 for n in c])))
                            song_chords.append(song_chord)
                        all_chords.append(song_chords)
                        all_keys.append((key_sig, bass_note))
                    # this is for pitch class frequency data
                        all_notes = []
                        note_data = {}
                        song_percents = []
                        note_counter = 0
                        for xx in crd:
                            for yy in xx:
                                all_notes.append((yy.pitchClass - bass_note) % 12)
                                note_counter += 1
                        for xx in range(12):
                            note_data[xx] = 0
                        for xx in all_notes:
                            note_data[xx] += 1
                        for xx in range(12):
                            song_percents.append(note_data[xx] / len(all_notes) * 100)
                        pitch_frequency_data.append(song_percents)
                        try:
                            print('completed', x.metadata.title, 'in ',time.time()-sst, ' seconds')
                        except AttributeError:
                            print('completed', 'notitle', 'in ',time.time()-sst, ' seconds')
                        all_corpus_labels.append((the_key[0].sharps, bass_note))

            elif type(x) is stream.Opus:
                print('we have an Opus!')
                for y in x:
                    song_chords = []
                    crd = y.chordify().flat.getElementsByClass(chord.Chord)
                    if len(x.flat.getElementsByClass(key.KeySignature)) > 0:
                        key_sig = y.flat.getElementsByClass(key.KeySignature)[0].sharps
                        bass_note = crd[-1].bass().pitchClass
                        modal_type = []
                        if modality == 'tonal':
                            for c in tonal_major_dict.keys():
                                modal_type.append(c)
                            for c in tonal_minor_dict.keys():
                                modal_type.append(c)
                                #                     modal_type.append(c for c in tonal_minor_dict.keys())
                        elif modality == 'modal':
                            for c in modal_major:
                                modal_type.append(c)
                            for c in modal_minor:
                                modal_type.append(c)
                        if (key_sig, bass_note) in modal_type:
                            print((key_sig, bass_note), 'is in there')
                            for c in crd:
                                song_chord = []
                                note_bass = (c.bass().pitchClass - bass_note) % 12
                                song_chord.append(note_bass)
                                song_chord.append(sorted(set([(n.pitchClass - bass_note - note_bass) % 12 for n in c])))
                                song_chords.append(song_chord)
                            all_chords.append(song_chords)
                            all_keys.append((key_sig, bass_note))
                        # this is for pitch class frequency data
                            all_notes = []
                            note_data = {}
                            song_percents = []
                            note_counter = 0
                            for xx in crd:
                                for yy in xx:
                                    all_notes.append((yy.pitchClass - bass_note) % 12)
                                    note_counter += 1
                            for xx in range(12):
                                note_data[xx] = 0
                            for xx in all_notes:
                                note_data[xx] += 1
                            for xx in range(12):
                                song_percents.append(note_data[xx] / len(all_notes) * 100)
                            pitch_frequency_data.append(song_percents)
                            try:
                                print('completed', x.metadata.title, 'in ',time.time()-sst, ' seconds')
                            except AttributeError:
                                print('completed', 'notitle', 'in ',time.time()-sst, ' seconds')

    start = time.time()
    if type(unparsed) is str:

        for root, dirs, files in os.walk(unparsed):
            for file_name, x in zip(files, range(len(files))):
#                 print(file_name, '...', x + 1, 'of', len(files)+1, '...')
                if file_name.endswith('.krn'):
                    #                     print('it is a krn')
                    path = os.path.join(root, file_name)
                    st = time.time()
                    print('parsing', file_name)
                    all_parsed.append(converter.parse(path))
#                     parser(path)
                    fin = time.time()
                    print('parsed', len(all_parsed), 'of', len(unparsed), 'in', fin - st, 'seconds')
    else:
        for x in unparsed:
            st = time.time()
#             print('parsing', x)
            all_parsed.append(converter.parse(x))
#             parser(x)
            fin = time.time()
            print('parsed', len(all_parsed), 'of', len(unparsed), 'in', fin - st, 'seconds')
    print('entire process took:', time.time() - start, 'seconds')
    parser(all_parsed)
    joblib.dump((all_chords, all_keys), 'pickles/%s_continuo.pkl' % corpus_name, compress=9)
    joblib.dump((pitch_frequency_data, all_keys), 'pickles/%s_notes.pkl' % corpus_name, compress=9)

    print('ALL DONE!!! :) :) :)')

In [4]:
#crd_data(path_to_folder, mode, corpus_name)
#for mode: If songs composed before 1700, use 'modal', otherwise, use 'tonal'. 
#for corpus_name: name it something simple that is similar to the name of the folder.
A = corpus.getComposer('bach')
data_converter(A, 'tonal', 'bach')

#this will print out 'finished 1 of 20...'. It's possible to see "finished 56 out of 20". This is normal; some files
    #contain more than one song. 

#nothing will be saved until this is finished. It should print "ALL DONE!" at the very end unless something went wrong.
#it may take hours, so these are best run overnight

parsed 1 of 426 in 0.07833194732666016 seconds
parsed 2 of 426 in 0.050308942794799805 seconds
parsed 3 of 426 in 0.06383490562438965 seconds
parsed 4 of 426 in 0.06691789627075195 seconds
parsed 5 of 426 in 0.06722593307495117 seconds
parsed 6 of 426 in 0.06831216812133789 seconds
parsed 7 of 426 in 0.05838894844055176 seconds
parsed 8 of 426 in 0.17583298683166504 seconds
parsed 9 of 426 in 0.05685615539550781 seconds
parsed 10 of 426 in 0.08181190490722656 seconds
parsed 11 of 426 in 0.1286020278930664 seconds
parsed 12 of 426 in 0.06187582015991211 seconds
parsed 13 of 426 in 0.04814505577087402 seconds
parsed 14 of 426 in 0.047772884368896484 seconds
parsed 15 of 426 in 0.04146409034729004 seconds
parsed 16 of 426 in 0.04640483856201172 seconds
parsed 17 of 426 in 0.1781938076019287 seconds
parsed 18 of 426 in 0.057405948638916016 seconds
parsed 19 of 426 in 0.05252718925476074 seconds
parsed 20 of 426 in 0.0473477840423584 seconds
parsed 21 of 426 in 0.13058781623840332 seconds
p

parsed 173 of 426 in 0.0451810359954834 seconds
parsed 174 of 426 in 0.04617190361022949 seconds
parsed 175 of 426 in 0.05656075477600098 seconds
parsed 176 of 426 in 0.02677321434020996 seconds
parsed 177 of 426 in 0.026561975479125977 seconds
parsed 178 of 426 in 0.04232382774353027 seconds
parsed 179 of 426 in 0.053446054458618164 seconds
parsed 180 of 426 in 0.044524192810058594 seconds
parsed 181 of 426 in 0.04150509834289551 seconds
parsed 182 of 426 in 0.02408123016357422 seconds
parsed 183 of 426 in 0.04547119140625 seconds
parsed 184 of 426 in 1.1001579761505127 seconds
parsed 185 of 426 in 0.038568973541259766 seconds
parsed 186 of 426 in 0.13861298561096191 seconds
parsed 187 of 426 in 0.029860973358154297 seconds
parsed 188 of 426 in 0.03814697265625 seconds
parsed 189 of 426 in 0.040846824645996094 seconds
parsed 190 of 426 in 0.028978824615478516 seconds
parsed 191 of 426 in 0.05623292922973633 seconds
parsed 192 of 426 in 0.037161827087402344 seconds
parsed 193 of 426 in

parsed 340 of 426 in 0.0446009635925293 seconds
parsed 341 of 426 in 0.04502296447753906 seconds
parsed 342 of 426 in 0.04745793342590332 seconds
parsed 343 of 426 in 0.06021618843078613 seconds
parsed 344 of 426 in 0.04738497734069824 seconds
parsed 345 of 426 in 0.03438711166381836 seconds
parsed 346 of 426 in 0.03532695770263672 seconds
parsed 347 of 426 in 0.08224916458129883 seconds
parsed 348 of 426 in 0.044573068618774414 seconds
parsed 349 of 426 in 0.03875112533569336 seconds
parsed 350 of 426 in 0.059268951416015625 seconds
parsed 351 of 426 in 0.09093928337097168 seconds
parsed 352 of 426 in 0.03471207618713379 seconds
parsed 353 of 426 in 0.03830575942993164 seconds
parsed 354 of 426 in 0.04308581352233887 seconds
parsed 355 of 426 in 0.03609919548034668 seconds
parsed 356 of 426 in 0.040255069732666016 seconds
parsed 357 of 426 in 0.05852317810058594 seconds
parsed 358 of 426 in 0.041680097579956055 seconds
parsed 359 of 426 in 0.06424498558044434 seconds
parsed 360 of 426

/Users/daniel/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:38: Music21DeprecationWarning: Note._getPitchClass was deprecated on May 2014 and will disappear at or after May 2016. use pitch.pitchClass instead


(-2, 7) is in there
completed bwv10.7.mxl in  0.6853668689727783  seconds
(-1, 2) is in there
completed bwv101.7.mxl in  0.5370030403137207  seconds
(2, 11) is in there
completed bwv103.6.mxl in  1.0748789310455322  seconds
(3, 9) is in there
completed bwv104.6.mxl in  0.9554359912872314  seconds
(2, 11) is in there
completed bwv108.6.mxl in  1.1931641101837158  seconds
(2, 2) is in there
completed bwv11.6.mxl in  2.0052762031555176  seconds
(2, 11) is in there
completed bwv110.7.mxl in  0.8576300144195557  seconds
(0, 9) is in there
completed bwv111.6.mxl in  1.4125149250030518  seconds
(1, 7) is in there
completed bwv112.5-sc.mxl in  1.9264698028564453  seconds
(1, 7) is in there
completed bwv112.5.mxl in  1.2599799633026123  seconds
(2, 11) is in there
completed bwv113.8.mxl in  0.9939510822296143  seconds
(-2, 7) is in there
completed bwv114.7.mxl in  0.9466290473937988  seconds
(1, 7) is in there
completed bwv115.6.mxl in  1.0562191009521484  seconds
(3, 9) is in there
completed b

(1, 7) is in there
completed bwv248.12-2.mxl in  1.0771958827972412  seconds
(0, 0) is in there
completed bwv248.17.mxl in  0.5931611061096191  seconds
(1, 7) is in there
completed bwv248.23-2.mxl in  2.915243148803711  seconds
(1, 7) is in there
completed bwv248.23-s.mxl in  0.5167727470397949  seconds
(1, 7) is in there
completed bwv248.33-3.mxl in  0.637397289276123  seconds
(3, 6) is in there
completed bwv248.35-3.mxl in  0.8139770030975342  seconds
(3, 6) is in there
completed bwv248.35-3c.mxl in  1.0306119918823242  seconds
(-1, 5) is in there
completed bwv248.42-4.mxl in  19.10617184638977  seconds
(-1, 5) is in there
completed bwv248.42-s.mxl in  1.5969421863555908  seconds
(3, 9) is in there
completed bwv248.46-5.mxl in  0.7693114280700684  seconds
(3, 9) is in there
completed bwv248.53-5.mxl in  0.8457498550415039  seconds
(1, 7) is in there
completed bwv248.59-6.mxl in  1.163268804550171  seconds
(2, 2) is in there
completed bwv248.64-6.mxl in  129.5601830482483  seconds
(2,

(0, 0) is in there
completed bwv367.mxl in  0.6975009441375732  seconds
(-1, 5) is in there
completed bwv368.mxl in  1.303595781326294  seconds
(1, 4) is in there
completed bwv369.mxl in  0.8802721500396729  seconds
(3, 9) is in there
completed bwv37.6.mxl in  0.9585521221160889  seconds
(1, 7) is in there
completed bwv373.mxl in  0.9069640636444092  seconds
(-1, 2) is in there
completed bwv374.mxl in  1.2599737644195557  seconds
(1, 7) is in there
completed bwv375.mxl in  0.6242480278015137  seconds
(3, 9) is in there
completed bwv376.mxl in  0.6357851028442383  seconds
(2, 2) is in there
completed bwv377.mxl in  0.7433867454528809  seconds
(1, 7) is in there
completed bwv378.mxl in  1.0301430225372314  seconds
(1, 7) is in there
completed bwv379.mxl in  0.8434350490570068  seconds
(-3, 3) is in there
completed bwv380.mxl in  0.9278442859649658  seconds
(1, 4) is in there
completed bwv381.mxl in  1.0533878803253174  seconds
(-3, 0) is in there
completed bwv384.mxl in  0.57411789894104

(4, 4) is in there
completed Es ist das Heil uns kommen her in  0.7815420627593994  seconds
(1, 7) is in there
completed An Wasserfluessen Babylon in  1.385429859161377  seconds
(-1, 5) is in there
completed Christus, der ist mein Leben in  0.5551919937133789  seconds
(3, 9) is in there
completed Nun lob, mein Seel, den Herren in  2.0507256984710693  seconds
(-4, 5) is in there
completed Freuet euch, ihr Christen alle in  1.2590219974517822  seconds
(1, 7) is in there
completed Ermuntre dich, mein schwacher Geist in  0.950829029083252  seconds
(0, 0) is in there
completed Jesu, nun sei gepreiset in  1.6524150371551514  seconds
(0, 9) is in there
completed Puer natus in Bethlehem in  0.8828320503234863  seconds
(0, 9) is in there
completed Allein zu dir, Herr Jesu Christ in  1.107105016708374  seconds
(1, 7) is in there
completed O Herre Gott, dein goettlich Wort in  0.9760839939117432  seconds
(-1, 2) is in there
completed Christ lag in Todesbanden in  1.0439140796661377  seconds
(1, 4

In [ ]:
#do it in parallel
import ipyparallel as ipp


rc = ipp.Client()
dview = rc[:]
print(dview)

dview.execute('from music21 import *')
dview.execute('import pickle, time, os')
dview.execute("from sklearn.externals import joblib")

@dview.parallel(block=True)
def parallel_data_converter(number):
    corpus_dict = {0: {'name': 'victoria_masses', 'path': '/home/daniel/Desktop/ElvisVictoria/ElVIS Victoria Masses/', 'modality': 'modal'}, 
                   1: {'name': 'victoria_motets', 'path': '/home/daniel/Desktop/ElvisVictoria/ElViS Motets/', 'modality': 'modal'},
                   2: {'name': 'monteverdi', 'path': corpus.getComposer('monteverdi'), 'modality': 'modal'}}
    tonal_major_dict = {(0, 0): 'CM', (1, 7): 'GM', (2, 2): 'DM', (3, 9): 'AM', (4, 4): 'EM', (5, 11): 'BM', (6, 6): 'F♯M',
               (-1, 5): 'FM', (-2, 10): 'B♭M', (-3, 3): 'E♭M', (-4, 8): 'A♭M', (-5, 1): 'D♭M', (-6, 6): 'G♭M'}
    tonal_minor_dict = {(0, 9): 'am', (1, 4): 'em', (2, 11): 'bm', (3, 6): 'f♯m', (4, 1): 'c♯m', (5, 8): 'g♯m',
                        (6, 3): 'd♯m', (-1, 2): 'dm', (-2, 7): 'gm', (-3, 0): 'cm', (-4, 5): 'fm', (-5, 10): 'b♭m',
                        (-6, 3): 'e♭m'}
    modal_major = [('f', 5), ('n', 0), ('f', 0), ('n', 7), ('f', 10), ('n', 5),
                   (-1, 5), (0, 0), (-1, 0), (0, 7), (-1, 10), (0, 5)]
    modal_minor = [('n', 2), ('n', 9), ('f', 2), ('f', 7), ('n', 4),
                   (0, 2), (0, 9), (-1, 2), (-1, 7), (0, 4)]
    # this function converts .krn or .xml files into the format we need
    def data_converter(unparsed, modality, corpus_name):
        pitch_frequency_data = []
        all_parsed = []
        all_keys = []
        all_chords = []

        def parser(parsed_songs):
            for x in all_parsed:
                if type(x) is stream.Score:
        #             print('we have a score!')
                    song_chords = []
                    crd = x.chordify().flat.getElementsByClass(chord.Chord)
                    if len(x.flat.getElementsByClass(key.KeySignature)) > 0:
                        key_sig = x.flat.getElementsByClass(key.KeySignature)[0].sharps
                        bass_note = crd[-1].bass().pitchClass
                        modal_type = []
                        if modality == 'tonal':
                            for c in tonal_major_dict.keys():
                                modal_type.append(c)
                            for c in tonal_minor_dict.keys():
                                modal_type.append(c)
                                #                 modal_type.append(c for c in tonal_minor_dict.keys())
                        elif modality == 'modal':
                            for c in modal_major:
                                modal_type.append(c)
                            for c in modal_minor:
                                modal_type.append(c)
                                #             print(modal_type, (key_sig, bass_note))
#                         if (key_sig, bass_note) in modal_type: #DO THIS IF ONLY MOLLIS/DURUS WANTED
#                             print((key_sig, bass_note), 'is in there')
                        for c in crd:
                            song_chord = []
                            note_bass = (c.bass().pitchClass - bass_note) % 12
                            song_chord.append(note_bass)
                            song_chord.append(sorted(set([(n.pitchClass - bass_note - note_bass) % 12 for n in c])))
                            song_chords.append(song_chord)
                        all_chords.append(song_chords)
                        all_keys.append((key_sig, bass_note))
                    # this is for pitch class frequency data
                        all_notes = []
                        note_data = {}
                        song_percents = []
                        note_counter = 0
                        for xx in crd:
                            for yy in xx:
                                all_notes.append((yy.pitchClass - bass_note) % 12)
                                note_counter += 1
                        for xx in range(12):
                            note_data[xx] = 0
                        for xx in all_notes:
                            note_data[xx] += 1
                        for xx in range(12):
                            song_percents.append(note_data[xx] / len(all_notes) * 100)
                        pitch_frequency_data.append(song_percents)
                        print(len(pitch_frequency_data), len(all_keys))
                        print((key_sig, bass_note))
                    #                 all_corpus_labels.append((the_key[0].sharps, bass_note))

                elif type(x) is stream.Opus:
                    print('we have an Opus!')
                    for y in x:
                        song_chords = []
                        crd = y.chordify().flat.getElementsByClass(chord.Chord)
                        if len(x.flat.getElementsByClass(key.KeySignature)) > 0:
                            key_sig = y.flat.getElementsByClass(key.KeySignature)[0].sharps
                            bass_note = crd[-1].bass().pitchClass
                            modal_type = []
                            if modality == 'tonal':
                                for c in tonal_major_dict.keys():
                                    modal_type.append(c)
                                for c in tonal_minor_dict.keys():
                                    modal_type.append(c)
                                    #                     modal_type.append(c for c in tonal_minor_dict.keys())
                            elif modality == 'modal':
                                for c in modal_major:
                                    modal_type.append(c)
                                for c in modal_minor:
                                    modal_type.append(c)
#                             if (key_sig, bass_note) in modal_type: #DO THIS IF MOLIS/DURUS WANTED
#                                 print((key_sig, bass_note), 'is in there')
                            for c in crd:
                                song_chord = []
                                note_bass = (c.bass().pitchClass - bass_note) % 12
                                song_chord.append(note_bass)
                                song_chord.append(sorted(set([(n.pitchClass - bass_note - note_bass) % 12 for n in c])))
                                song_chords.append(song_chord)
                            all_chords.append(song_chords)
                            all_keys.append((key_sig, bass_note))
                        # this is for pitch class frequency data
                            all_notes = []
                            note_data = {}
                            song_percents = []
                            note_counter = 0
                            for xx in crd:
                                for yy in xx:
                                    all_notes.append((yy.pitchClass - bass_note) % 12)
                                    note_counter += 1
                            for xx in range(12):
                                note_data[xx] = 0
                            for xx in all_notes:
                                note_data[xx] += 1
                            for xx in range(12):
                                song_percents.append(note_data[xx] / len(all_notes) * 100)
                            pitch_frequency_data.append(song_percents)
                            print(len(pitch_frequency_data), len(all_keys))

        start = time.time()
        if type(unparsed) is str:

            for root, dirs, files in os.walk(unparsed):
                for file_name, x in zip(files, range(len(files))):
    #                 print(file_name, '...', x + 1, 'of', len(files)+1, '...')
                    if file_name.endswith('.krn') or file_name.endswith('.mid'):
                        #                     print('it is a krn')
                        path = os.path.join(root, file_name)
                        st = time.time()
                        all_parsed.append(converter.parse(path))
    #                     parser(path)
                        fin = time.time()
    #                     print('finished', len(all_chords), 'of', len(unparsed), 'in', fin - st, 'seconds')
        else:
            for x in unparsed:
                st = time.time()
                all_parsed.append(converter.parse(x))
    #             parser(x)
                fin = time.time()
    #             print('finished', len(all_chords), 'of', len(unparsed), 'in', fin - st, 'seconds')
        print('entire process took:', time.time() - start, 'seconds')
        parser(all_parsed)
        joblib.dump((all_chords, all_keys), 'pickles/%s_continuo.pkl' % corpus_name, compress=9)
        joblib.dump((pitch_frequency_data, all_keys), 'pickles/%s_notes.pkl' % corpus_name, compress=9)

        print('ALL DONE!!! :) :) :)')
    
    data_converter(corpus_dict[number]['path'], corpus_dict[number]['modality'], corpus_dict[number]['name'])

In [ ]:
parallel_data_converter.map(range(3))

In [ ]:
j = converter.parse('/home/daniel/Desktop/ElVIS Victoria Masses/Missa-Alma-Redemptoris-mater_Agnus-Dei_Victoria-Tomas-Luis-de_file3.mid')

In [ ]:
j.show('text')

In [ ]:
b = corpus.getComposer('bach')

In [ ]:
print(len(b))
converter.parse(b[156]).show('text')

In [ ]:
s = 'this is the %sst try' %1.1

In [ ]:
s

In [1]:
a = [1, 2, 3]
b = [2, 3, 4]

In [2]:
import numpy as np

In [3]:
np.mean(a, b)

TypeError: tuple indices must be integers or slices, not list

In [1]:
from sklearn.externals import joblib

In [5]:
x = joblib.load('pickles/alfabeto_roots.pickle')

In [6]:
x

([[7,
   2,
   4,
   7,
   0,
   7,
   9,
   8,
   9,
   7,
   0,
   7,
   0,
   7,
   0,
   7,
   9,
   8,
   9,
   7,
   9,
   5,
   7,
   9,
   2,
   7,
   0,
   9,
   5,
   0,
   7,
   0],
  [0, 7, 10, 3, 10, 5, 0, 5, 10, 3, 10, 3, 5, 7, 0],
  [3,
   5,
   7,
   10,
   3,
   10,
   0,
   5,
   8,
   7,
   0,
   7,
   0,
   10,
   7,
   3,
   0,
   2,
   7,
   8,
   3,
   8,
   2,
   7,
   0,
   7,
   0],
  [0,
   7,
   0,
   9,
   5,
   7,
   0,
   7,
   9,
   4,
   0,
   2,
   7,
   10,
   0,
   5,
   7,
   0,
   10,
   9,
   2,
   5,
   4,
   5,
   7,
   0],
  [3, 10, 0, 3, 8, 10, 3, 0, 6, 7, 0, 7, 0, 5, 0, 5, 7, 0],
  [0,
   2,
   7,
   0,
   5,
   10,
   5,
   10,
   0,
   5,
   8,
   9,
   2,
   7,
   0,
   7,
   0,
   2,
   7,
   0,
   5,
   7,
   0,
   8,
   9,
   7,
   0,
   2,
   0,
   2,
   7,
   8,
   9,
   2,
   7,
   0,
   9,
   2,
   7,
   5,
   7,
   0],
  [3, 8, 0, 10, 0, 5, 7, 0, 2, 7, 0, 8, 10, 0, 5, 7, 8, 7, 0],
  [0, 7, 9, 5, 0, 7, 9, 5, 7, 0, 5, 2, 7, 9, 9, 4, 